# Geometric manipulation

## Upscaling and resizing

It is possible to upscale, downscale and stretch images with OpenCV.

In [15]:
import cv2

img = cv2.imread("samples/tree.png")
img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
cv2.imshow("Downscaled image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
stretched_img = cv2.resize(img, None, None, fx=3, fy=0.5)
cv2.imshow("Stretched image", stretched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

It is also possible to express the desired manipulation through a matrix:

In [17]:
import numpy as np

M = np.float32([[0, 1, 0],
              [1, 0, 0]])
stretched_img = cv2.warpAffine(img, M, img.shape[:-1])
cv2.imshow("Stretched image", stretched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Rotation

In [18]:
rows, cols = img.shape[0], img.shape[1]
R = cv2.getRotationMatrix2D((cols//2, rows//2), 30, 1)
rotated_img = cv2.warpAffine(img, R, (cols, rows))
cv2.imshow("Rotated image", rotated_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Translation

In [19]:
tx = 0
ty = 0
t = [[1, 0, tx],
     [0, 1, ty]]

## A panorama function and generic affine transformations

In generic transformations, we need at least 3 points to infer the original function.  
They preserve parallelism.

In [20]:
pts_1 = np.float32([[135, 45],
                    [385, 45],
                    [135, 230]])

pts_2 = np.float32([[135, 45],
                    [385, 45],
                    [150, 230]])

M = cv2.getAffineTransform(pts_1, pts_2)    #this is an affine transformation that, given the first
                                            #set of points, returns the second one
panorama = cv2.warpAffine(pts_1, M, (800, 800))
print(M)
#cv2.imshow("Chosen set of points", panorama)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

[[ 1.          0.08108108 -3.64864865]
 [ 0.          1.          0.        ]]


## Scanner application

First of all, we need to import the picture.

In [21]:
img = cv2.imread("samples/gerry.png")
cv2.imshow("Original", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Now, we can take the corner coordinates:

* tl - 28, 227
* bl- 131, 987
* tr - 572, 149
* br - 730, 860

In [22]:
src_points = np.array([
    [28, 227],
    [131, 987],
    [572, 149],
    [730, 860]
],  np.float32)

Some debugging on coordinates position:

In [23]:
points = img.copy()
points = cv2.circle(points, np.asarray(src_points[3], np.int32), 10, (0, 0, 255), -1)

points = cv2.resize(points, None, None, fx = 0.7, fy = 0.7)
cv2.imshow("Scanned image", points)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
#We want to rescale it in a 600x800 image
dst_points = np.array(
    [
        [0, 0],
        [0, 800],
        [600, 0],
        [600, 800],
    ], np.float32
)

In [25]:
#get the transformation matrix M: we use perspectiveTransorm since it
#is not just an affine transformation, the contours of the image are not
#parallel in the original
M = cv2.getPerspectiveTransform(src_points, dst_points)
out_img = cv2.warpPerspective(img, M, (600, 800))
cv2.imshow("Scanned image", out_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### A different implementation of the scanner

Let's create an interface in order to get the starting points.

In [26]:
#Starting steps

img = cv2.imread("samples/gerry.png")
img = cv2.resize(img, None, None, fx = 0.7, fy = 0.7)
img_copy = img.copy()

src_points = []
dst_points = np.array(
    [
        [0, 0],
        [0, 600],
        [800, 0],
        [800, 600],
    ], np.float32
)

In [27]:
#Creating the window for selecting the starting points

#defining the callback function on the event of clicking the left button
def onClick(event, x, y, flags, param):
    if event == cv2.EVENT_FLAG_LBUTTON:
        if len(src_points) < 4:
            src_points.append([x, y])           #pay attention to the square brackets
            cv2.circle(img_copy, (x, y), 10, (0, 0, 255), -1)
            cv2.imshow("Img", img_copy)

cv2.namedWindow('Img')
#the next function takes input of the mouse as the callback for the previously created window
cv2.setMouseCallback("Img", onClick)
#Show the image
cv2.imshow("Img", img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows

#Compute the matrix M
#We first need to convert the points to the correct format

src_float = np.array(src_points, dtype=np.float32)
M = cv2.getPerspectiveTransform(src_float, dst_points)

#Get the final image
out_img = cv2.warpPerspective(img, M, (800, 600))
cv2.imshow("Img", out_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Projection

A projection requires 4 points to be inferred.